In [ ]:
!nvidia-smi

Tue Apr 19 12:29:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |    266MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Main Libraries

In [ ]:
pip install -q -U tensorflow-addons

     |████████████████████████████████| 1.1 MB 14.9 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import datetime
import math
import itertools 
import pickle
import time
import os
import sys
import shutil
import keras
import glob
import cv2
import random
import textwrap
import scipy.ndimage as ndimage
import seaborn as sns

from sklearn.model_selection import train_test_split
from math import ceil, floor
from pathlib import Path
from tqdm import tqdm
from sklearn import metrics
from tqdm.notebook import tqdm
from IPython.display import clear_output
from PIL import Image, ImageDraw, ImageFont

# Setup Colab

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Unzip Datasets

In [ ]:
!ls "/content/drive/My Drive/ColabNotebooks/Datasets"
!unzip "/content/drive/My Drive/ColabNotebooks/Datasets/hyperkvasir-labelled-data.zip"
!unzip "/content/drive/MyDrive/ColabNotebooks/Datasets/hyper-kvasir-unlabeled-images.zip"

# Initialize

In [ ]:
labeled_data_root = pathlib.Path('/content/hyperkvasir-labelled-data')
unlab_data_root = pathlib.Path('/content/unlabeled-images')

project_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Primary directory
log_dir = "./logs/{}/{}".format(project_time, "labelled_model")
pathlib.Path(log_dir).mkdir(parents=True, exist_ok=True) # log directory

cache_dir = "./cache"
pathlib.Path(cache_dir).mkdir(parents=True, exist_ok=True) # cache directory

# Split Dataset

In [ ]:
def split_dataset():

    train_split = float(0.7)
    test_split = float(0.7)+float(0.15)
    random.seed(1500)
    
    # listdir() returns a list containing the names of the entries in the directory
    class_names = os.listdir(labeled_data_root)
    num_classes = len(class_names)
    num_samples = len(list(Path(labeled_data_root).glob('*/*')))

    image_folders = [os.path.join(labeled_data_root, class_name) for class_name in class_names]

    train_folders = [os.path.join('labeled_data', 'train', name) for name in class_names]
    test_folders = [os.path.join('labeled_data', 'test', name) for name in class_names]
    val_folders = [os.path.join('labeled_data', 'val', name) for name in class_names]

    for folder in train_folders:
      Path(folder).mkdir(parents=True, exist_ok=True)

    for folder in test_folders:
      Path(folder).mkdir(parents=True, exist_ok=True)
                              
    for folder in val_folders:
      Path(folder).mkdir(parents=True, exist_ok=True)

    tqdm_img = tqdm(total=num_samples, desc='Images', position=0)

    for idx, directory in enumerate(image_folders):

        random.seed(1500)
        filenames = os.listdir(directory)
        filenames.sort() # make sure that the filenames have a fixed order before shuffling
        random.shuffle(filenames) # shuffles the ordering of filenames (deterministic given the chosen seed)
        num_samples = len(filenames)

        filenames = np.array(filenames)
        filenames_split = np.split(filenames, [floor(num_samples*train_split), floor(num_samples*test_split)])
        
        ds = {'train': filenames_split[0],
              'test': filenames_split[1],
              'val': filenames_split[2]}

        for split in ds:
            output = os.path.join('labeled_data', split, directory.split("/")[-1])
            for filename in ds[split]:
                filename = os.path.join(directory, filename)
                resize_and_save(filename, output)
                tqdm_img.update(1)

In [ ]:
def resize_and_save(filename, output_dir):
    # Used to resize the image contained in `filename` and save it to the `output_dir`
    image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(image, (128, 128), interpolation=cv2.INTER_LINEAR)
    path = os.path.join(output_dir, filename.split('/')[-1])
    cv2.imwrite(path, image)

In [ ]:
split_dataset()

Images:   0%|          | 0/10662 [00:00<?, ?it/s]

In [ ]:
BATCH_SIZE = 8
CLASS_NAMES = None
NUMBER_CLASSES = 0
DATASET_SIZE_TRAIN = None
DATASET_SIZE_TEST = None
DATASET_SIZE_VAL = None
DATASET_SIZE_UNLAB = None
STEPS_TRAIN = None
STEPS_TEST = None
STEPS_VAL = None
AUTOTUNE = tf.data.experimental.AUTOTUNE

# function to load input image, preprocess it and get label
def load_label_images(image_path):
    # read the image from database, decode it, convert the data type to
	  # floating point, and then resize it
    image_raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_raw, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])

    # parse the class label from the file path
    str_targets = tf.strings.split(image_path, os.path.sep)[-2]
    label_integer = tf.reduce_min(tf.where(tf.equal(str_targets, CLASS_NAMES)))
    label = tf.dtypes.cast(label_integer, tf.int32)

    return image, label

def build_datasets():

    global CLASS_NAMES
    global NUMBER_CLASSES
    global DATASET_SIZE_TRAIN
    global DATASET_SIZE_TEST
    global DATASET_SIZE_VAL
    global STEPS_TRAIN
    global STEPS_TEST
    global STEPS_VAL

    labeled_data_root = pathlib.Path('/content/labeled_data')
    np.random.seed(seed=1500)

    class_names = np.array([item.name for item in labeled_data_root.glob('train/*')])

    CLASS_NAMES = class_names
    NUMBER_CLASSES = len(class_names)

    model_class_names = list(class_names)
    handle = open(log_dir+"/model_class_names.pkl", 'wb')
    pickle.dump(model_class_names, handle)
    
    ds = {'train': str(labeled_data_root/'train'/'*/*.*g'),
          'test': str(labeled_data_root/'test'/'*/*.*g'),
          'val': str(labeled_data_root/'val'/'*/*.*g')}

    DATASET_SIZE_TRAIN = len(list(glob.glob(ds["train"])))
    DATASET_SIZE_TEST = len(list(glob.glob(ds["test"])))
    DATASET_SIZE_VAL = len(list(glob.glob(ds["val"])))
    
    for split in ds:
        ds[split] = tf.data.Dataset.list_files(ds[split],
                                    shuffle=True,
                                    seed=tf.constant(1500, tf.int64)
                                    )
    
    # Map labels with the images from tf.dataset to create "train", "test", "val" datasets
    for split in ds:
        ds[split] = ds[split].map(load_label_images, num_parallel_calls=AUTOTUNE)
    
    clean_train = ds["train"] # Clean dataset of the "train" split
    
    print("[INFO] loading the dataset...")

    # training dataset split will go through a resample, suffle, repeat, data augment,
    # and prefetched phase
    ds["train"] = resample(ds["train"], len(class_names))
    ds["train"] = (
      ds["train"]
      .shuffle(buffer_size=3000, seed=tf.constant(1500, tf.int64))
      .repeat()
      .map(augment_ds(ds["train"]), num_parallel_calls=AUTOTUNE)
      .batch(BATCH_SIZE, drop_remainder=False)
      .prefetch(buffer_size=AUTOTUNE)
    )

    # testing dataset split will go through a caching, suffle, repeat,
    # and prefetched phase
    cache_string = "{}/{}_{}".format(cache_dir, 128, "test")
    ds["test"] = (
      ds["test"]
      .cache(cache_string)
      .shuffle(buffer_size=3000, seed=tf.constant(1500, tf.int64))
      .repeat()
      .batch(BATCH_SIZE, drop_remainder=False)
      .prefetch(buffer_size=AUTOTUNE)
    )

    # testing dataset split will go through a caching, suffle, repeat,
    # and prefetched phase
    cache_string = "{}/{}_{}".format(cache_dir, 128, "val")
    ds["val"] = (
      ds["val"]
      .cache(cache_string)
      .shuffle(buffer_size=3000, seed=tf.constant(1500, tf.int64))
      .repeat()
      .batch(BATCH_SIZE, drop_remainder=False)
      .prefetch(buffer_size=AUTOTUNE)
    )

    print("[INFO] train, test and val datasets created")

    STEPS_TRAIN = DATASET_SIZE_TRAIN//BATCH_SIZE
    STEPS_TEST = DATASET_SIZE_TEST//BATCH_SIZE
    STEPS_VAL = DATASET_SIZE_VAL//BATCH_SIZE

    ds["clean_train"] = clean_train
    
    return ds

In [ ]:
def count_class_samples(ds, num_classes):
    count = np.zeros(num_classes) # returns a new array with zeros according to the class count
    for img, lab in ds:
        count[lab] += 1
    return count

Resample Dataset

In [ ]:
def resample(ds, class_count):

    datasets = []
    cache_dir = './cache/{}_resampled_{}/'.format(128, 'train')
    pathlib.Path(cache_dir).mkdir(parents=True, exist_ok=True) # create cache directory
    
    for i in range(class_count): # class_count = 23
        data = ds.filter(lambda img, lab: lab==i)
        data = data.cache(cache_dir+'{}_ds'.format(i))
        
        data = data.repeat()
        datasets.append(data)
    
    target_dist = [ 1.0/class_count ] * class_count
    balanced_ds = tf.data.experimental.sample_from_datasets(datasets, weights=target_dist, seed=1500)
    
    return balanced_ds

Data Augmentation

In [ ]:
def augment_ds(ds):

    tf.random.set_seed(1500) # global seed
    
    def image_augment(img, label):

        degree = tf.random.normal([])*360
        img = tfa.image.rotate(img, degree * math.pi / 180, interpolation='BILINEAR')
        img = tf.image.resize_with_crop_or_pad(img, 138, 138)
        img = tf.image.random_crop(img, (128, 128, 3), seed=1500)
        img = tf.image.random_flip_left_right(img, seed=1500)
        img = tf.image.random_flip_up_down(img, seed=1500)
        img = tf.image.random_brightness(img, max_delta=0.2, seed=1500)
        img = tf.image.random_saturation(img, lower=0.5, upper=1.5, seed=1500)
        img = tf.image.random_contrast(img, lower=0.5, upper=1.5, seed=1500)

        # The random_* ops do not necessarily clamp.
        # Make sure image is still in [0, 1]
        img = tf.clip_by_value(img, 0.0, 1.0)
        return img, label
    
    return image_augment

In [ ]:
ds = build_datasets()

[INFO] loading the dataset...
[INFO] train, test and val datasets created


# Load unlabelled dataset

In [ ]:
def load_unlab_images(image_path):
    # read the image from database, decode it, convert the data type to
	  # floating point, and then resize it
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])

    # obtain filename
    filename = tf.strings.split(image_path, os.path.sep)[-1]
    return image, filename

def create_unlab_ds():

  global DATASET_SIZE_UNLAB

  unlab_data_root = pathlib.Path('/content/unlabeled-images/images')

  DATASET_SIZE_UNLAB = len(list(unlab_data_root.glob('*.*g')))

  files_string = str(unlab_data_root/'*.*g')
  list_ds_unlabeled = tf.data.Dataset.list_files(
          files_string, 
          shuffle=True, 
          seed=tf.constant(1500, tf.int64)
  )
  
  unlab_ds = list_ds_unlabeled.map(load_unlab_images, num_parallel_calls=AUTOTUNE)

  print ("Loaded {} images into unlabeled_ds.".format(DATASET_SIZE_UNLAB))
  
  return unlab_ds

In [ ]:
# First iteration only
ds["unlab"] = create_unlab_ds()
datasets_bin = [count_class_samples(ds["clean_train"], NUMBER_CLASSES)]
ds["mix_train"] = ds["clean_train"]

Loaded 99417 images into unlabeled_ds.


# Define the model used for labelled data

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPool2D, Multiply, Concatenate, Input, Dropout, ZeroPadding2D
from tensorflow.keras import backend as K
from keras.layers import concatenate

def create_model():

    base_model = tf.keras.applications.resnet.ResNet101(
        weights='imagenet',    
        include_top=False, 
        input_shape=(128, 128, 3)
    )
    
    base_model.trainable = True
    
    model = Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(NUMBER_CLASSES, activation="softmax"))
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
    )
    
    model.summary()
    
    return model


In [ ]:
model = create_model()

# Train the model for labelled data

In [ ]:
start = time.time()

history = model.fit(
        ds["train"],
        steps_per_epoch = STEPS_TRAIN,
        epochs = 15,
        validation_data = ds["val"],
        validation_steps = STEPS_VAL,
        validation_freq = 1,
        verbose = 1
)

print ("Total time:", (np.round(time.time() - start, 4)/60), "minutes.")

In [ ]:
print(history.history.keys())

dict_keys(['loss', 'sparse_categorical_accuracy', 'val_loss', 'val_sparse_categorical_accuracy'])


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks (1)/Experiments/InceptionResNetV2+ResNet50/Exp1

/content/drive/MyDrive/Colab Notebooks (1)/Experiments/InceptionResNetV2+ResNet50/Exp1


In [ ]:
cd /content

/content


In [ ]:
def save_model():
  handle = open(log_dir+"/history.pkl", 'wb')
  pickle.dump(history.history, handle)
  model.save(log_dir+'/model.h5')

## Model Evaluation

In [ ]:
def calculate_true_and_predicted_cat(model, ds):

  # Using test data to evaluate the model
  test_evaluate = model.evaluate(ds["test"], steps=STEPS_TEST) # steps since tf.dataset
  print("\nTest Accuracy: ", test_evaluate)

  # Find true_categories and predicted_categories
  eval_ds = ds["test"].unbatch().take(DATASET_SIZE_TEST)
  eval_ds = eval_ds.as_numpy_iterator()
  eval_ds = np.array(list(eval_ds))
  true_categories = list(eval_ds[:,1]) 
  eval_images = np.stack(eval_ds[:,0], axis=0) # eval_ds[:,0] = give me the 0th index of all the rows in eval_ds

  # Create predictions and predicted_categories
  predictions = model.predict(eval_images)
  predicted_categories = [np.argmax(pred) for pred in predictions]

  return true_categories, predicted_categories

def using_evaluation_metrics(history, true_categories, predicted_categories):

    sns.set()
    
    # Plot Accuracy
    plot_accuracy(history)
    # Plot Loss
    plot_loss(history)

    print('\nAccuracy score is :', metrics.accuracy_score(true_categories, predicted_categories))
    print('Precision score is :', metrics.precision_score(true_categories, predicted_categories, average='weighted'))
    print('Recall score is :',metrics.recall_score(true_categories, predicted_categories, average='weighted'))
    print('F1 Score is :', metrics.f1_score(true_categories, predicted_categories,average='weighted'))
    print('Cohen Kappa Score :', metrics.cohen_kappa_score(true_categories, predicted_categories))
    print("MCC :", metrics.matthews_corrcoef(true_categories, predicted_categories))
    
    # Print the Classification report
    print("\nClassification Report: ")
    report = metrics.classification_report( y_true=true_categories, y_pred=predicted_categories,
                labels=range(NUMBER_CLASSES),
                target_names=CLASS_NAMES,
                digits=3,
                zero_division=0)
    print(report)

def plot_confusion_matrix(cm, names=None):

    print('Confusion Matrix')
    
    plt.figure(figsize=(20,20))

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    plt.colorbar()

    tick_marks = np.arange(NUMBER_CLASSES)
    plt.xticks(tick_marks, CLASS_NAMES, rotation=90)
    plt.yticks(tick_marks, CLASS_NAMES)

    # normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm = np.around(cm, decimals=2)
    cm[np.isnan(cm)] = 0.0
    print("Normalized confusion matrix")

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True Categories')
    plt.xlabel('Predicted Categories')
    plt.savefig(log_dir+"/confusion_matrix.pdf", format="pdf")
    plt.show()

def plot_accuracy(history):
  
  plt.plot(history.history['sparse_categorical_accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_sparse_categorical_accuracy'], label='Validation Accuracy')
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(loc='upper left')
  plt.show()

def plot_loss(history):
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(loc='upper left')
  plt.show()

In [ ]:
true_categories, predicted_categories = calculate_true_and_predicted_cat(model, ds)
using_evaluation_metrics(history, true_categories, predicted_categories)

# Plot the Confusion Matrix
cm = metrics.confusion_matrix(true_categories, predicted_categories)
plot_confusion_matrix(cm, CLASS_NAMES)

save_model()

# Generate pseudo labels on unlabeled images using the previously built model

### Generate pseudo labels for the unlabelled dataset

In [ ]:
findings = {
    "prediction_value_list": [], 
    "label_index_list": [], 
    "img_name_list": []
    }

In [ ]:
def generate_pseudo_labels(unlab_ds, model):

    # Use the unlabeled dataset to generate new psuedo labels
    start = time.time()
    findings_count = 0

    # Show tqdm bars
    predicting_bar = tqdm(total=DATASET_SIZE_UNLAB, desc='Predicting', position=0, initial=0)
    findings_bar = tqdm(total=DATASET_SIZE_UNLAB, desc='Findings', position=1, bar_format='{desc}:{bar}{n_fmt}', initial=0)
    
    # Iterates the images one by one
    for tot_cnt, (image,path) in enumerate(unlab_ds, start=0):
      if tot_cnt > 20000:
          break
              
      img = np.expand_dims(image, 0)
      prediction = model.predict(img) # returns numpy array(s) of predictions.
      highest_pred = np.max(prediction) # returns the maximum value

      # Check if the highest_pred is more than the threshold value
      if highest_pred > 0.85:
          prediction_index = np.argmax(prediction).astype(np.uint8) # returns the index of the max element

          findings["label_index_list"].append(prediction_index)
          findings["prediction_value_list"].append(highest_pred)
          findings["img_name_list"].append(path)
              
          findings_count += 1
          findings_bar.update(1)
      predicting_bar.update(1)

    # Save the pseudo labels
    handle = open(log_dir+"/pseudo_labels.pkl", 'wb')
    pickle.dump(findings, handle)
        
    print ("\nTotal run time: {:.1f} min.".format( (time.time() - start)/60 ))
    print ("Found {} new samples in unlabeled_ds after looking at {} images.".format(findings_count, tot_cnt))
        
    return findings

In [ ]:
findings = generate_pseudo_labels(ds["unlab"], model)

Predicting:   0%|          | 0/99417 [00:00<?, ?it/s]

Findings:          0


Total run time: 29.3 min.
Found 8253 new samples in unlabeled_ds after looking at 20001 images.


### Sort the lists from confidence levels highest to lowest



In [ ]:
sorted_list = list(zip(findings["prediction_value_list"], findings["label_index_list"], findings["img_name_list"]))
sorted_list.sort(key=lambda x: x[0], reverse=True)

pred_sorted = [row[0] for row in sorted_list]
lab_sorted = [row[1] for row in sorted_list]
name_sorted = [row[2] for row in sorted_list]

pseudo_sorted = {
    "prediction_value_list": pred_sorted,
    "label_index_list": lab_sorted,
    "img_name_list": name_sorted
}

### Create new dataset by combining train data and new findings

In [ ]:
unlab_data_root = pathlib.Path('/content/unlabeled-images/images')

In [ ]:
def create_new_dataset(ds, pseudo_sorted, datasets_bin):

  new_findings, added_samples = resample_labeled_dataset(pseudo_sorted, datasets_bin[-1])

  # create a Dataset whose elements are slices of the new findings
  findings_tensor = tf.data.Dataset.from_tensor_slices(new_findings)
  ds["mix_train"] = ds["mix_train"].concatenate(findings_tensor) # Combine the train dataset with new findings

  # count class samples of the combined dataset
  datasets_bin.append(count_class_samples(ds["mix_train"], NUMBER_CLASSES))

  return datasets_bin, added_samples

def read_unlab_images(tensor, folder):
    tensor = tensor.numpy().decode("utf-8")
    image = tf.io.read_file("{}/{}".format(folder, tensor))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    return image

def resample_labeled_dataset(pseudo, orig_dist):

    # Add the new findings to respective classes
    added = {}
    total_added = 0
        
    print ('Resample')
    print ("-"*50)

    new_findings = ([], [])
    new_findings_filepaths = []
    lab_arr = np.asarray(pseudo["label_index_list"], dtype=np.uint8)

    for class_idx in range(NUMBER_CLASSES):
        # how many samples already in this class
        in_count = orig_dist[class_idx]
        print('in_count:', in_count)

        indexes = np.where(lab_arr==class_idx)[0]
        num_new_findings = len(indexes)

        count = 0
        for count, idx in enumerate(indexes, start=1):
            if in_count >= 1000:
                count -= 1 # reduce by one cuz of enumerate updates index early
                break
            tensor = pseudo["img_name_list"][idx]
            image = read_unlab_images(tensor, unlab_data_root)
            
            new_findings[0].append(image)                             # image
            new_findings[1].append(pseudo["label_index_list"][idx])         # label
            new_findings_filepaths.append(pseudo["img_name_list"][idx]) # filepath
            in_count += 1
            
        total_added += count

        print ("{:27}: added {}/{} samples".format(CLASS_NAMES[class_idx], count, num_new_findings))
            
        added[CLASS_NAMES[class_idx]] = [count, num_new_findings]
    
    print ("-"*50)
    text = "Added a total of {} samples to the training dataset. New dataset size is {}."
    print (text.format(total_added, DATASET_SIZE_TRAIN + total_added))
        
    return new_findings, new_findings_filepaths

In [ ]:
datasets_bin, added_samples = create_new_dataset(ds, pseudo_sorted, datasets_bin)

original_unlab_size = DATASET_SIZE_UNLAB
new_unlab_size = DATASET_SIZE_UNLAB - len(added_samples)
original_train_size = int(np.sum(datasets_bin[-2]))
new_train_size = int(np.sum(datasets_bin[-1]))

print("\nFinal Dataset Details:")
print("Newly added Samples: ", len(added_samples))
print("Unlabelled dataset size changed from {} to {} samples".format(original_unlab_size, new_unlab_size))
print("Labelled dataset size changed from {} to {} samples".format(original_train_size, new_train_size))

# Update dataset sizes
DATASET_SIZE_UNLAB -= len(added_samples)
DATASET_SIZE_TRAIN = new_train_size
STEPS_TRAIN = new_train_size//BATCH_SIZE 

##Use the new dataset to train a pseudo model

In [ ]:
# Pseudo directory
log_dir = "./logs/{}/{}".format(project_time, "pseudo_model")
pathlib.Path(log_dir).mkdir(parents=True, exist_ok=True) # log directory

ds["train"] = resample(ds["mix_train"], NUMBER_CLASSES)
ds["train"] = (
  ds["train"]
  .shuffle(buffer_size=3000, seed=tf.constant(1500, tf.int64))
  .repeat()
  .map(augment_ds(ds["train"]), num_parallel_calls=AUTOTUNE)
  .batch(BATCH_SIZE, drop_remainder=False)
  .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
new_model = create_model()

In [ ]:
start_time = time.time()

history = model.fit(
    ds["train"],
    steps_per_epoch = STEPS_TRAIN, 
    epochs = 15,
    validation_data = ds["val"],
    validation_steps = STEPS_VAL,
    validation_freq = 1,
    verbose = 1
)
print ("Time spent on training: {:.2f} minutes.".format(np.round(time.time() - start_time)/60))

### Model Evaluation

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks (1)/Experiments/InceptionResNetV2+ResNet50/Exp1

/content/drive/MyDrive/Colab Notebooks (1)/Experiments/InceptionResNetV2+ResNet50/Exp1


In [ ]:
model.save('hyperkvasir_InceptionResNetV2+ResNet50_student.h5')

In [ ]:
cd /content

/content


In [ ]:
true_categories, predicted_categories = calculate_true_and_predicted_cat(model, ds)
using_evaluation_metrics(history, true_categories, predicted_categories)

# Plot the Confusion Matrix
cm = metrics.confusion_matrix(true_categories, predicted_categories)
plot_confusion_matrix(cm, CLASS_NAMES)